In [5]:
import tensorflow as tf

import urllib
import random


from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop
from shutil import copyfile

In [6]:
import os
import zipfile

# Extract the archive
zip_ref = zipfile.ZipFile("./catsdogs.zip", 'r')
zip_ref.extractall("./tmp/")
zip_ref.close()

BadZipFile: ignored

In [ ]:
print("Number of cat images:", len(os.listdir('/tmp/catsdogs/PetImages/Cat/')))

Create folders to store the training and test data

* There will be a training and testing folder 
* There will be subfolders for each of these (one for cats and one for dogs)

In [ ]:
try:
    os.mkdir('./tmp/cats-v-dogs')
    os.mkdir('./tmp/cats-v-dogs/training')
    os.mkdir('./tmp/cats-v-dogs/testing')
    os.mkdir('./tmp/cats-v-dogs/training/cats')
    os.mkdir('./tmp/cats-v-dogs/training/dogs')
    os.mkdir('./tmp/cats-v-dogs/testing/cats')
    os.mkdir('./tmp/cats-v-dogs/testing/dogs')
except OSError:
    pass

##Split data into training and test sets 
* The following code will first check if an image is empty (zero length)
* Of the files that aren't empy, it puts 90% of the data into the training set and 10% into the test set

In [ ]:
import random 
from shutil import copyfile

def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
  files = []
  
  for filename in os.listdir(SOURCE):
    file = SOURCE + filename
    if os.path.getsize(file) > 0:
      files.append(filename)
    else: 
      print(filename + 'zero length, so ignoring')

  training_length = int(len(files) * SPLIT_SIZE)
  testing_length = int(len(files) - training_length)

  shuffled_set = random.sample(files, len(files))
  training_set = shuffled_set[0: training_length]
  testing_set = shuffled_set[training_length:]

  for filename in training_set:
    this_file = SOURCE + filename
    destination = TRAINING + filename
    copyfile(this_file, destination)

  for filename in testing_set:
    this_file = SOURCE + filename
    destination = TESTING + filename
    copyfile(this_file, destination)

CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
TRAINING_CATS_DIR = "/tmp/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "/tmp/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
TRAINING_DOGS_DIR = "/tmp/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "/tmp/cats-v-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

# Expected output
# 666.jpg is zero length, so ignoring
# 11702.jpg is zero length, so ignoring

In [ ]:

print("Number of training cat images", len(os.listdir('/tmp/cats-v-dogs/training/cats/')))
print("Number of training dog images", len(os.listdir('/tmp/cats-v-dogs/training/dogs/')))
print("Number of testing cat images", len(os.listdir('/tmp/cats-v-dogs/testing/cats/')))
print("Number of testing dog images", len(os.listdir('/tmp/cats-v-dogs/testing/dogs/')))

# expected output
# Number of training cat images 11250
# Number of training dog images 11250
# Number of testing cat images 1250
# Number of testing dog images 1250

##Data Augmentation (try adjusting the parameters)

Use `ImageDataGenerator` to perform data augmentation 

Play around with it to achieve 99.9% validation accuracy or better 

In [ ]:
TRAINING_DIR = '/tmp/cats-v-dogs/training'

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 40, 
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   fill_mode = 'nearest')

train_generator = train_datagen.flow_from_directory(VALIDATION_DIR, 
                                                    batch_size = 100,
                                                    class_mode = 'binary',
                                                    target_size = (150,150))

##Get and Prep the Model 

Use `InceptionV3`

* Load the pre-trained weights of the model 
* Freeze the existing layers so that they're not trained on the downstream task
* Get a reference to the last layer since there are some layers that need to be added after the last one

In [8]:
weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weights_file = "inception_v3.h5"
urllib.request.urlretrieve(weights_url, weights_file)

('inception_v3.h5', <http.client.HTTPMessage at 0x7f3e0c930a90>)

In [9]:
#Instantiate the model 
pre_trained_model = InceptionV3(input_shape = (150, 150, 3),
                                include_top = False,
                                weights = None)

#Load the pretrained weights 
pre_trained_model.load_weights(weights_file)

#Free the layers 
for layer in pre_trained_model.layers:
  layer.trainable = False

last_layer = pre_trained_model.get_layer('mixed7')
print('Last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

Last layer output shape: (None, 7, 7, 768)


##Add layers 

Add some layers to train the cats and dogs data on:

* `Flatten`: Will take the output of the `last_layer` and flatten it into a vector
* `Dense`: Add a dense layer with `relu` activation
* `Dense`: Add a dense layer with `sigmoid` activation. This will scale the output range to 0 to 1 and will allow the output to be predicted as cats or dogs 

In [ ]:
x = layers.Flatten()(last_output)

x = layers.Dense(1024, activation='relu')(x)

x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

## Train the model 

Compile the model and then train it using `model.fit()`

* Feel free to adjust the number epochs or setup a callback to end the training once a certain level of accuracy has been achieved 


In [ ]:
#Compile the model 
model.compile(optimizer = RMSprop(lr = 1e-4),
              loss = 'binary_crossentropy',
              metrics = ['acc'])

#Train the model 
history = model.fit(train_generator,
                    validation_data = validation_generator,
                    epochs = 2, #test value
                    verbose = 1)

##Visualize the training and validation accuracy

In [ ]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

#Retrieve a list of list results on training and test data (sets for each training epoch)
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc)) #Get number of epochs 

#Plot the training and validation accuracy per epoch
plt.plot(epochs, acc, 'r', 'Training Accuracy')
plt.plot(epochs, val_acc, 'b', 'Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.figure()

##Predict on a test image

Upload any image and have the model predict whether it's a dog or a cat

In [ ]:
import numpy as np
form google.colab import files 
from keras.preprocessing import image

uploaded = files.upload()
plt.figure(figsize = (15, 30))
n_images = len(uploaded.keys())

for idx, fn in enumerate(uploaded.keys()):

  #Predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size = (150, 150))
  plt.subplot(1, n_images, idx + 1)
  plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis = 0)


  image_tensor = np.vstack([x])
  classes = model.predict(image_tensor)

  print(classes)
  print(classes[0])

  if classes[0] > 0.5:
    print(fn + ' is a dog')
    plt.title(fn + ' is a dog')
  else: 
    print(fn + ' is a cat')
    plt.title(fn + ' is a cat')

  plt.show()